In [1]:
# Learned how to use json to get the current citibke status
import requests
import json
import pandas as pd

# Public json url from Citibike
stations_url = 'https://feeds.citibikenyc.com/stations/stations.json'

# Request the response and parse the json
resp = requests.get(url=stations_url)
data = resp.json()

In [2]:
# Show a sample of the json dicts
# The Citibike Station json returns a list of dicts
data['stationBeanList'][0]

{'id': 298,
 'stationName': '3 Ave & Schermerhorn St',
 'availableDocks': 12,
 'totalDocks': 35,
 'latitude': 40.68683208,
 'longitude': -73.9796772,
 'statusValue': 'In Service',
 'statusKey': 1,
 'availableBikes': 23,
 'stAddress1': '3 Ave & Schermerhorn St',
 'stAddress2': '',
 'city': '',
 'postalCode': '',
 'location': '',
 'altitude': '',
 'testStation': False,
 'lastCommunicationTime': '2019-07-07 08:01:17 PM',
 'landMark': ''}

In [3]:
# Process the list of dictionary into DataFrame
station_df = pd.DataFrame(data['stationBeanList'])

In [4]:
# Show the list of columns in the DataFrame
station_df.columns

Index(['altitude', 'availableBikes', 'availableDocks', 'city', 'id',
       'landMark', 'lastCommunicationTime', 'latitude', 'location',
       'longitude', 'postalCode', 'stAddress1', 'stAddress2', 'stationName',
       'statusKey', 'statusValue', 'testStation', 'totalDocks'],
      dtype='object')

In [5]:
# Preview the DataFrame
station_df.head(5)

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,23,12,,298,,2019-07-07 08:01:17 PM,40.686832,,-73.979677,,3 Ave & Schermerhorn St,,3 Ave & Schermerhorn St,1,In Service,False,35
1,,17,11,,307,,2019-07-07 08:02:21 PM,40.714275,,-73.989900,,Canal St & Rutgers St,,Canal St & Rutgers St,1,In Service,False,30
2,,6,43,,358,,2019-07-07 08:01:03 PM,40.732916,,-74.007114,,Christopher St & Greenwich St,,Christopher St & Greenwich St,1,In Service,False,50
3,,12,15,,426,,2019-07-07 08:02:35 PM,40.717548,,-74.013221,,West St & Chambers St,,West St & Chambers St,1,In Service,False,29
4,,24,5,,432,,2019-07-07 07:59:35 PM,40.726218,,-73.983799,,E 7 St & Avenue A,,E 7 St & Avenue A,1,In Service,False,31


In [6]:
station_df = station_df[['id', 
                         'stationName',
                         'availableBikes', 
                         'availableDocks', 
                         'totalDocks',
                         'statusValue']]

In [7]:
station_df.sort_values(by='availableBikes', 
                             ascending=False)
station_df.head(10)

,id,stationName,availableBikes,availableDocks,totalDocks,statusValue
0,298,3 Ave & Schermerhorn St,23,12,35,In Service
1,307,Canal St & Rutgers St,17,11,30,In Service
2,358,Christopher St & Greenwich St,6,43,50,In Service
3,426,West St & Chambers St,12,15,29,In Service
4,432,E 7 St & Avenue A,24,5,31,In Service
5,438,St Marks Pl & 1 Ave,29,17,47,In Service
6,487,E 20 St & FDR Drive,13,21,34,In Service
7,504,1 Ave & E 16 St,15,38,53,In Service
8,3092,Berry St & N 8 St,15,12,27,In Service
9,3255,8 Ave & W 31 St,6,12,19,In Service


In [8]:
# Create a boolean array mask where there are no bikes
# Make sure the station is in service right now.
in_service_mask = (station_df['statusValue'] == 'In Service')

# Active station only
active_station = station_df[in_service_mask]

# Find out which station has no bike avaliable
zero_bike_mask = (active_station['availableBikes'] == 0)

# Apply the zero_bike_mask
zero_bike = active_station[zero_bike_mask]

# Sample the output
zero_bike.sample(5)

,id,stationName,availableBikes,availableDocks,totalDocks,statusValue
433,3233,E 48 St & 5 Ave,0,39,39,In Service
195,450,W 49 St & 8 Ave,0,64,67,In Service
635,3507,Park Ave & E 124 St,0,35,36,In Service
418,3206,Hilltop,0,26,26,In Service
609,3467,W Broadway & Spring Street,0,41,42,In Service


# Question: How many stations are without bike?

In [9]:
# Get the count of stations without bikes
num_of_stations_without_bike = zero_bike['id'].count()
print('There are {0:,} stations without bike!'.format(
    num_of_stations_without_bike))

There are 37 stations without bike!


In [10]:
# Find out which station has no dock avaliable
zero_dock_mask = (active_station['availableDocks'] == 0)

# Apply the zero_dock_mask
zero_dock = active_station[zero_dock_mask]

zero_dock.sample(5)

,id,stationName,availableBikes,availableDocks,totalDocks,statusValue
815,3758,George St & Wilson Ave,19,0,19,In Service
487,3312,1 Ave & E 94 St,39,0,39,In Service
794,3715,Driggs Ave & N 9 St,30,0,31,In Service
220,481,S 3 St & Bedford Ave,25,0,25,In Service
79,295,Pike St & E Broadway,24,0,24,In Service


# Question: How many stations are without a dock?

In [11]:
num_of_stations_without_dock = zero_dock['id'].count()
print('There are {0:,} stations without dock!'.format(
    num_of_stations_without_dock))

There are 34 stations without dock!


In [12]:
no_dock = num_of_stations_without_dock
no_bike = num_of_stations_without_bike
total = active_station.shape[0] - no_dock - no_bike

In [13]:
arr = pd.Series([no_dock, no_bike, total],
                dtype='int64',
                index=['No Dock', 'No Bike', 'Active (Has Both)'])

In [14]:
arr.plot(kind='bar', rot=0,
         ylim=(0, 900),
         title='Current Citibike System Health')

In [15]:
# Motivated to plot these station onto a map!

In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

KeyError: 'PROJ_LIB'